In [ ]:
# Dependencies and Setup
import pandas as pd

# File to Load (*Remember to Change These)
file = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
games_df = pd.read_csv(file)
games_df.columns 
#I just want to print out all of the columns so I have them in a list and can refer back when I want. 
# This is more for my preference

#Found this formatting method that I am going to try and use further down when cleaning my data:
#https://towardsdatascience.com/style-pandas-dataframe-like-a-master-6b02bf6468b0

In [ ]:
#CHALLENGE 1: Player Count --DONE
#Display the total number of players
#I just want to display an example of the DF. 
#I am going to drop all on my duplicates SN's then double check the new DF count for SNs. 
nodups_df = games_df.drop_duplicates("SN",inplace = False) 
#inplace = False will return a copy of the DF but with the duplicates dropped
nodups_df['SN'].count()
# I want to create a new DF here to displau total unique players
totalplayers_df=pd.DataFrame({'Total Players': [nodups_df['SN'].count()]})
#print out new DF
totalplayers_df

In [ ]:
#CHALLENGE 2: Purchasing Analysis (Total) --DONE
#Run basic calculations to obtain number of unique items, average price, etc
#Create a summary data frame to hold the results
    #Optional: give the displayed data cleaner formatting
    #Display the summary data frame
# I need to establish the formulas as variables and place them into a dictionary. I plan to start groupbys
#further down.
items = games_df["Item Name"].nunique() #Saw this trick here: https://stackoverflow.com/questions/38309729/count-unique-values-with-pandas-per-groups
prices = round(games_df["Price"].mean(),2)
purchases = games_df["Purchase ID"].count()
revenue = games_df["Price"].sum()
#I now want to create a dictionary with new column titles for each new variable; the variables will be the 
#the values. I will then create a new dataframe and assign the new dictionary to it. This is similar to 
#the breakout where we created a dictionary of states and abbreviations. 
#I need to f string and the new format method for inserting special characters that Mindie showed us.
newdict = [{'Number of Unique Items': items, "Average Price": (f'${prices}'), 
           "Number of Purchases": purchases,  "Total Revenue": revenue}]
newdata_df = pd.DataFrame(newdict)
newdata_df.style.format({"Total Revenue":"${:,}"})

In [ ]:
# CHALLENGE 3: GENDER DEMOGRAPHICS: --DONE
    #Percentage and Count of Male Players
    #Percentage and Count of Female Players
    #Percentage and Count of Other / Non-Disclosed
# Since this is broken down to look at gender I will need to do a groupby gender here from the no duplicates DF
#This is my variable for total unique count
# My variable for the total of unique gamers by gender. I am placing this outside of my groupby to reference later, if need be.
genders = nodups_df['Gender'].count()
# My groupby here.
groupgender = nodups_df.groupby(["Gender"]) 
groupgender_count = groupgender['SN'].count()
genderpercent = (groupgender_count/genders)*100
gend_df = pd.DataFrame({"Total Count": groupgender_count, "Percentage of Players":genderpercent})
# Clean the data
gend_df.head().style.format({'Percentage of Players':"{:.2f}%"})


In [ ]:
#CHALLENGE 4: PURCHASING ANALYSIS (GENDER)
#Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. 
#by gender
#Create a summary data frame to hold the results
    #Optional: give the displayed data cleaner formatting
    #Display the summary data frame
groupgender2 = games_df.groupby(['Gender']) 
groupgender_count2 = groupgender2.SN.count() 
ggtotal = groupgender2.Price.sum()
gendaveragepurchase = ggtotal / groupgender_count2
totalavg = ggtotal / gend_df['Total Count']
gend2_df = pd.DataFrame({"Purchase Count": groupgender_count2,
                         "Average Purchase Price": gendaveragepurchase,
                        "Total Purchase Value":ggtotal,
                        'Avg Total Purchase per Person': totalavg})
gend2_df.head().style.format({'Average Purchase Price': '${:,.2f}',
                              'Total Purchase Value': '${:,.2f}',
                              'Avg Total Purchase per Person':'${:,.2f}'})

In [ ]:
# CHALLENGE 5: AGE DEMOGRAPHICS
    #Establish bins for ages - DONE
    #Categorize the existing players using the age bins. Hint: use pd.cut() - DONE
    #Calculate the numbers and percentages by age group - DONE
    #Create a summary data frame to hold the results - DONE
    #Optional: round the percentage column to two decimal points
    #Display Age Demographics Table

In [ ]:
#Create my bins and ages: the bins will the be age ranges and the names will be the titles of the age ranges
agebins = [0, 9, 14, 19, 24, 29, 34, 39, 99]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]
#I want to use my nodups_df, create a new column and essentially append the column of 'Age' and have them placed
#into the age buckets I created. The age buckets do not have a title in the homework so I am making them = '' 
#I am using the hint and doing the pd.cut method
nodups_df[''] = pd.cut(nodups_df['Age'], agebins, labels=group_names)
#Here I will need to groupby my newly created column--this is like the gender count above so I will also need
#some variables within the groupby to get my count and percentage
groupbyage_df = nodups_df.groupby('')
tc = groupbyage_df["Age"].count()
percp = tc/genders*100
agedemo_df = pd.DataFrame({'Total Count': tc,'Percentage of Players':percp})
#Clean the column
agedemo_df.style.format({'Percentage of Players':"{:.2f}%"})

In [ ]:
# CHALLENGE 6: Purchasing Analysis (Age)
    #Bin the purchase_data data frame by age
    #Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
    #Create a summary data frame to hold the results
    #Optional: give the displayed data cleaner formatting
    #Display the summary data frame

In [ ]:
#This activity is going to be similar to gender purchase analysis with the same breakdown of variables within 
# a groupby. 
agebins2 = [0, 9, 14, 19, 24, 29, 34, 39, 99]
group_names2 = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]
games_df['Age Ranges'] = pd.cut(games_df['Age'], agebins2, labels=group_names2)
# I will need to do another groupby 
groupbyage2 = games_df.groupby('Age Ranges')
# Variables and formulas from my Groupby
agetc2 = groupbyage2['Age'].count()
agetpv = groupbyage2['Price'].sum()
ageavepurch = agetpv/agetc2
totalageavepurch = agetpv / tc
agedemo2_df = pd.DataFrame({'Purchase Count': agetc2,"Average Purchase Price": ageavepurch,
                            'Total Purchase Value':agetpv,'Avg Total Purchase per Person':totalageavepurch})
#Clean up the dataframe
agedemo2_df.style.format({"Average Purchase Price":"${:,.2f}",'Total Purchase Value':"${:,.2f}", 
                          'Avg Total Purchase per Person':"${:,.2f}"})

In [ ]:
# CHALLENGE 7:  Top Spenders
    #Run basic calculations to obtain the results in the table below
    #Create a summary data frame to hold the results
    #Sort the total purchase value column in descending order
    #Optional: give the displayed data cleaner formatting
    #Display a preview of the summary data frame

In [ ]:
#Group by screen name and display values similar to the above purchase analysis
#sort the last column by descending
topgroup = games_df.groupby(['SN'])
topgrouppc = topgroup['Item Name'].count()
topgrouptotal = topgroup['Price'].sum()
topgroupavg = topgrouptotal / topgrouppc
topspender_df = pd.DataFrame({'Purchase Count': topgrouppc,'Average Purchase Price': topgroupavg,
                             'Total Purchase Value': topgrouptotal})
#Sort
topspender_df.sort_values(by='Total Purchase Value', ascending=False, inplace=True)
#Clean--I inserted bar graphs to give a better descending visualization of the data
#https://towardsdatascience.com/style-pandas-dataframe-like-a-master-6b02bf6468b0
topspender_df.head().style.format({'Average Purchase Price':'${:,.2f}','Total Purchase Value':'${:,.2f}'})\
    .bar(subset=['Total Purchase Value',],color ='lightgreen')

In [ ]:
# CHALLENGE 8: Most Popular Items
    #Retrieve the Item ID, Item Name, and Item Price columns
    #Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, 
    # and total purchase value
    #Create a summary data frame to hold the results
    #Sort the purchase count column in descending order
    #Optional: give the displayed data cleaner formatting
    #Display a preview of the summary data frame
#Very similar to the above; groupby and variables
mostpop = games_df.groupby(['Item ID','Item Name'])
itemprice = mostpop['Price'].mean()
mostpurch = mostpop['Item Name'].count()
totalpvalue = itemprice * mostpurch
mostpopitems_df = pd.DataFrame({'Purchase Count':mostpurch,
                            'Item Price': itemprice,
                            'Total Purchase Value': totalpvalue})
#Sort
mostpopitems_df.sort_values(by='Purchase Count', ascending = False,inplace=True) 
#Clean
mostpopitems_df.head().style.format({'Item Price':"${:,.2f}",'Total Purchase Value': '${:,.2f}'})\
    .bar(subset=['Purchase Count',],color ='lightgreen')

In [ ]:
# CHALLENGE 9:  Most Profitable Items
#Sort the above table by total purchase value in descending order
#Optional: give the displayed data cleaner formatting
#Display a preview of the data frame
#Same DF as above but I will need to sort by 'Total Purchase Value'
mostpopitems_df.sort_values(by='Total Purchase Value', ascending = False, inplace=True) 
#Clean
mostpopitems_df.head().style.format({'Total Purchase Value':"${:,.2f}",'Item Price':"${:,.2f}"})\
    .bar(subset=['Total Purchase Value',],color ='lightgreen')